# 数据清洗-用户信息

up 主信息：`../data/user/user_up_clean.csv`，评论者信息：`../data/user/user_com_clean.csv`, 所有相关用户信息整合：`../data/user/user_full_clean.csv`（该表为上面两个表信息整合，多了 `is_up` 字段区分up主信息还是评论者信息，若一个用户既是up主又是评论者，会被记录两次）

In [16]:
import pandas as pd
import numpy as np
import re
import os
import time
import datetime
from tqdm import tqdm

## Up 主信息

In [7]:
# up 主未清洗的信息
udf_up = pd.read_csv('../data/user/user_up.csv', encoding='ANSI')
udf_up.shape

(567, 34)

In [8]:
udf_up = udf_up.drop(['whisper', 'black', 'face', 'face_nft', 'rank', 'jointime', 'moral', 'silence', 'fans_medal', 'fans_badge'], axis = 1)
udf_up = udf_up.drop(['vip', 'pendant', 'nameplate', 'user_honour_info', 'is_followed', 'top_photo','profession', 'tags', 'official', ], axis = 1)
udf_up = udf_up.drop(['theme', 'sys_notice', 'live_room', 'school', 'series', 'is_senior_member', 'coins', 'birthday'], axis = 1)

In [9]:
udf_up['is_up'] = 1
udf_up.columns

Index(['mid', 'following', 'follower', 'name', 'sex', 'sign', 'level',
       'is_up'],
      dtype='object')

In [10]:
udf_up.to_csv("../data/user/user_up_clean.csv", index=False, header=True, encoding="utf-8")

## 评论者信息

In [14]:
path_r = "../data/reply/reply_by_bvid_clean"
path_r2 = "../data/reply/reply_by_bvid_clean_simple"
replies = os.listdir(path_r)

In [17]:
udf_list = []
for file in tqdm(replies):
    df = pd.read_csv(path_r + '\\' + file)
    udf = df[['mid', 'uname', 'sex', 'sign', 'level']].rename(columns={"uname":"name"})
    udf_list.append(udf)
    df = df.drop(['uname', 'sex', 'sign', 'level'], axis = 1)
    # save
    df.to_csv(path_r2 + "\\" + file, index=False, header=True, encoding="utf-8")
    # print(path_r2 + "\\" + file)

100%|██████████| 961/961 [00:19<00:00, 49.27it/s]


In [18]:
udf_follow = pd.read_csv('../data/user/user_com.csv')

In [22]:
udf = pd.concat(udf_list, axis = 0)
len(set(udf['mid'])), len(udf['mid'])

(254634, 908079)

In [23]:
udf = udf.drop_duplicates('mid', keep = 'first') # 去重

In [24]:
len(set(udf_follow['mid'])), len(udf_follow['mid'])

(254499, 254631)

In [26]:
udf_com = pd.merge(udf, udf_follow)
udf_com['is_up'] = 0
# udf_com

In [28]:
# 记录缺失信息用户
# umiss = pd.DataFrame({"mid":list(set(udf_follow['mid']).difference(set(udf['mid'])))}).merge(udf_follow)
# umiss.to_csv(r'D:\Ryanna\RUC_courses\Y1_2\市调大赛\data\user\user_miss.csv', index = False, header= True, encoding = "utf-8")

In [108]:
# 查看重复数值
# df1=udf.groupby(["mid"]).size()
# col=df1[df1>1].reset_index()[["mid"]]
# pd.merge(col,udf,on=["mid"]).to_csv("tmp.csv")

In [30]:
# 既是up主又是评论者人数
len(list(set(udf_up['mid']).intersection(set(udf_com['mid']))))

314

In [31]:
udf_full = pd.concat([udf_up, udf_com], axis = 0)

In [34]:
len(udf_full['mid']), len(set(udf_full['mid']))

(255198, 254752)

In [35]:
udf_up.to_csv('../data/user/user_up_clean.csv', index = False, header= True, encoding = "utf-8")
udf_com.to_csv('../data/user/user_com_clean.csv', index = False, header= True, encoding = "utf-8")
udf_full.to_csv('../data/user/user_full_clean.csv', index = False, header= True, encoding = "utf-8")